# Advent of Code Day 20: Race Condition
First, let's read the input file and prepare the maze data.

In [1]:
def read_maze(filename):
    with open(filename, 'r') as f:
        maze = [list(line.strip()) for line in f.readlines()]
    return maze

maze = read_maze('aoc20.txt')

Implement functions to find start and end positions and calculate shortest path

In [2]:
def find_positions(maze):
    start, end = None, None
    for y in range(len(maze)):
        for x in range(len(maze[0])):
            if maze[y][x] == 'S':
                start = (y, x)
            elif maze[y][x] == 'E':
                end = (y, x)
    return start, end

from collections import deque

def shortest_path(maze, start, end):
    queue = deque([(start, 0)])
    visited = {start}
    
    while queue:
        (y, x), steps = queue.popleft()
        if (y, x) == end:
            return steps
            
        for dy, dx in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            ny, nx = y + dy, x + dx
            if (0 <= ny < len(maze) and 0 <= nx < len(maze[0]) and 
                maze[ny][nx] != '#' and (ny, nx) not in visited):
                visited.add((ny, nx))
                queue.append(((ny, nx), steps + 1))
    
    return float('inf')

Function to check possible cheats and their time savings

In [3]:
def find_cheats(maze, base_time):
    start, end = find_positions(maze)
    savings = []
    height, width = len(maze), len(maze[0])
    
    for y1 in range(height):
        for x1 in range(width):
            if maze[y1][x1] == '#':
                continue
            
            path1 = shortest_path(maze, start, (y1, x1))
            if path1 == float('inf'):
                continue
                
            for y2 in range(max(0, y1-2), min(height, y1+3)):
                for x2 in range(max(0, x1-2), min(width, x1+3)):
                    if maze[y2][x2] == '#':
                        continue
                        
                    path2 = shortest_path(maze, (y2, x2), end)
                    if path2 != float('inf'):
                        total = path1 + path2 + abs(y2-y1) + abs(x2-x1)
                        if total < base_time:
                            savings.append(base_time - total)
                            if len(savings) >= 100:  # Early exit if we find enough savings
                                return savings
    
    return savings

Calculate the result

In [4]:
start, end = find_positions(maze)
base_time = shortest_path(maze, start, end)
savings = find_cheats(maze, base_time)

result = sum(1 for s in savings if s >= 100)
print(f"Number of cheats saving at least 100 picoseconds: {result}")

# Save result
with open('result.txt', 'w') as f:
    f.write(str(result))

Number of cheats saving at least 100 picoseconds: 22
